In [ ]:
# Setup: Add parent directory to Python path
import sys
import os

# Add the parent directory to the path so we can import pr_agent
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print(f"Python path configured. Working directory: {os.getcwd()}")

# PR Review Agent - Testing Notebook

This notebook allows you to test the PR review agent interactively.

## Prerequisites
1. Activate the virtual environment: `source venv/bin/activate`
2. Configure `.env` file in `pr_agent/.env` with:
   - `GOOGLE_CLOUD_PROJECT`
   - `GOOGLE_CLOUD_LOCATION`
   - `GITHUB_REPOSITORY`
   - `GITHUB_PR_NUMBER`
3. Authenticate with GCP: `gcloud auth application-default login`
4. Authenticate with GitHub: `gh auth login`

## Usage
Run the cells in order to:
- **Setup**: Configure Python path and load environment variables
- **Option 1**: Run the full `main()` function (uncomment to execute)
- **Option 2**: Run step-by-step to test each component individually

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv('../pr_agent/.env')

# Verify environment is configured
import os
print(f"GOOGLE_CLOUD_PROJECT: {os.getenv('GOOGLE_CLOUD_PROJECT')}")
print(f"GOOGLE_CLOUD_LOCATION: {os.getenv('GOOGLE_CLOUD_LOCATION')}")
print(f"GITHUB_REPOSITORY: {os.getenv('GITHUB_REPOSITORY')}")
print(f"GITHUB_PR_NUMBER: {os.getenv('GITHUB_PR_NUMBER')}")

In [ ]:
# Option 1: Run the full main() function
from pr_agent.main import main

# Uncomment to run the full workflow:
# main()

In [ ]:
# Option 2: Test individual parts step-by-step
from pr_agent.github_tools import get_pr_info, get_pr_diff, post_pr_comment

# Get configuration
repo = os.getenv('GITHUB_REPOSITORY')
pr_number = int(os.getenv('GITHUB_PR_NUMBER'))

print(f"Testing PR #{pr_number} in {repo}...")

# Step 1: Fetch PR metadata
pr_info = get_pr_info(repo, pr_number)
print(f"\nPR Title: {pr_info.get('title', 'N/A')}")
print(f"Author: {pr_info.get('author', {}).get('login', 'N/A')}")
print(f"Branch: {pr_info.get('headRefName', 'N/A')} -> {pr_info.get('baseRefName', 'N/A')}")

In [ ]:
# Step 2: Fetch PR diff
pr_diff = get_pr_diff(repo, pr_number)
print(f"Diff length: {len(pr_diff)} characters")
print(f"\nFirst 500 characters of diff:")
print(pr_diff[:500])

In [ ]:
# Step 3: Generate AI review
from google import genai

# Create review prompt
prompt = f"""Review this pull request:

Title: {pr_info.get('title', 'N/A')}

Description:
{pr_info.get('body', 'No description provided')}

Branch: {pr_info.get('headRefName', 'N/A')} -> {pr_info.get('baseRefName', 'N/A')}

Author: {pr_info.get('author', {}).get('login', 'N/A')}

Changes:
{pr_diff}

Provide your code review."""

# Initialize Gemini client
client = genai.Client(
    vertexai=True,
    project=os.getenv('GOOGLE_CLOUD_PROJECT'),
    location=os.getenv('GOOGLE_CLOUD_LOCATION')
)

system_instruction = """You are a code reviewer analyzing a pull request.

Review the PR for:
- Obvious bugs or logic errors
- Code quality issues (complexity, readability)
- Potential security issues
- Missing error handling
- Best practice violations

Provide a concise review summary with:
1. Overall assessment (Looks good / Needs work / Has issues)
2. Key findings (list 3-5 most important issues)
3. Positive observations (what's done well)

Keep feedback constructive and actionable.
Format your response in markdown for GitHub."""

print("Generating review with Gemini 2.5 Flash...")
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt,
    config=genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7
    )
)

review_text = response.text
print("\n" + "="*80)
print("REVIEW:")
print("="*80)
print(review_text)

In [ ]:
# Step 4: Post review comment to PR (uncomment to actually post)
# post_pr_comment(repo, pr_number, review_text)
# print(f"✓ Review posted to PR #{pr_number}")
# print(f"View at: https://github.com/{repo}/pull/{pr_number}")